In [ ]:
import srsly

In [ ]:
import os
os.chdir('..')

In [ ]:
from scripts import labeling_rules

In [ ]:
import spacy

nlp = spacy.load('en_core_web_trf')

In [ ]:
nlp.add_pipe("labeling_ruler")
nlp.add_pipe("labeling_extend_ruler")
nlp.add_pipe("expand_entities")

In [ ]:
items = srsly.read_jsonl('corpus/paragraphs_train.jsonl')
pars = []

In [ ]:
import pandas as pd
import numpy as np
df = pd.DataFrame(items)

In [ ]:
doc = nlp('The Taxonomy Regulation was published in the Official Journal of the European Union on 22 June 2020 and entered into force on 12 July 2020. It establishes the framework for the EU taxonomy by setting out four overarching conditions that an economic activity has to meet in order to qualify as environmentally sustainable. The Taxonomy Regulation establishes six environmental objectives Climate change mitigation Climate change adaptation The sustainable use and protection of water and marine resources The transition to a circular economy Pollution prevention and control The protection and restoration of biodiversity and ecosystems Different means can be required for an activity to make a substantial contribution to each objective. ')

In [ ]:
print([(ent.label_, ent) for ent in doc.ents])

In [ ]:
from IPython.display import HTML


In [ ]:
query = 'trillion sdg'
filtered_df = df[np.logical_and(*[df.text.str.lower().str.contains(word) for word in query.split(' ')])]
HTML(filtered_df.to_html())

In [ ]:
for i, par in enumerate(nlp.pipe((item['text'] for item in items))):
    if any((ent.label_ == 'IX_REFERENCE_VALUE' for ent in par.ents)):
        pars.append(par.text)

In [ ]:
len(pars) / (i + 1)

In [ ]:
from spacy.pipeline import EntityRuler
from spacy.language import Language

@Language.factory("ruler")
def make_optional_pattern_component(nlp: Language, name: str):
    ruler = EntityRuler(nlp, overwrite_ents=True)
    ruler.add_patterns([{
            "label": "IX_REFERENCE_VALUE",
            "pattern": [
                {"POS": "ADV"},
                {"POS": "ADJ"},
            ],
        },])
    return ruler

In [ ]:
nlp.disable_pipe('ner')

In [ ]:
nlp.add_pipe('ruler')

In [ ]:
for i, par in enumerate(nlp.pipe((item['text'] for item in items))):
    if any((ent.label_ == 'IX_REFERENCE_VALUE' for ent in par.ents)):
        print([ent for ent in par.ents if ent.label_ == 'IX_REFERENCE_VALUE'])
        pars.append(par.text)

In [ ]:
items = srsly.read_jsonl('corpus/reference_value_paragraphs_train_v1.jsonl')

In [ ]:
import re


def filter_digit_spans(items):
    span_count = 0
    for item in items:
        spans = []
        for span in item['spans']:
            span_texts = []
            for i in range(span['token_start'], span['token_end'] + 1):
                span_texts += [item['tokens'][i]['text']]
            span_text = ' '.join(span_texts)
            if span_text not in ['very small', 'lower', 'significantly lower', 'significantly higher', 'higher', 'approximately up to']:
                span_count += 1
                spans.append(span)
        item['spans'] = spans
        if spans:
            yield item
    print(span_count)

In [ ]:
srsly.write_jsonl('corpus/reference_value_paragraphs_train_v2.jsonl', filter_digit_spans(items))